## import

In [1]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
# from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
import xgboost as xgb
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier
import timeit
import matplotlib.pylab as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import ttest_ind_from_stats

In [2]:
# ttest_ind_from_stats(mean1=0.73314, std1=0.022, nobs1=30, mean2=0.6448, std2=0.0469, nobs2=30)

## Functions

In [6]:
####################################################################################################################
#################################################Parameters#########################################################
####################################################################################################################
location = 'Processed_Datasets'
file='Wine' #Wine #mushrooms #adult #diabetic_data #covtype


train_data='_Train_Data.csv'
test_data='_Test_Data.csv'
val_data = '_Val_Data.csv'
learner_model = 'XGB'

k = 3


# if file =='Wine':
#     IS_set=[0,1,2,3,4,7,8,9]
# #     IS_set = [1] 
# elif file == 'Normal':
#     IS_set = [7, 1, 4, 9]
# elif file == 'Mushrooms':
# #     IS_set = [1, 6, 0, 5, 3, 8, 9]
#     IS_set = [1, 2, 0, 5, 3, 8, 6, 7, 9]

In [7]:
####################################################################################################################
################################################# Data separate & Learner #########################################################
####################################################################################################################


# Function that separate the episode data into features and label
def data_separate (dataset,train):
    global X
    global y    
    if train ==1:
        X = dataset.iloc[:,0:dataset.shape[1]-2]  # all rows, all the features and no batch id
        y = dataset.iloc[:, dataset.shape[1]-1]  # all rows, label 
    else:
        X = dataset.iloc[:,0:dataset.shape[1]-1]  # all rows, all the features 
        y = dataset.iloc[:, dataset.shape[1]-1]  # all rows, label 
    return X,y



def Learner(X_train, X_test,y_train, y_test, learner_model):
    start = timeit.default_timer()
    global learner
    global y_pred
    
    if learner_model == 'DT':
        learner = tree.DecisionTreeClassifier(random_state=0)
        learner = learner.fit(X_train, y_train)   #remove batch_index column
        y_pred = learner.predict(X_test)  
    elif learner_model == 'GB':
        learner = GradientBoostingClassifier()
        learner = learner.fit(X_train, y_train)  
        y_pred = learner.predict(X_test)  
    elif learner_model == 'XGB':
        if file == 'diabetic_data':
#             label_encoder = LabelEncoder()
#             onehot_encoder = OneHotEncoder(sparse=False)
#             integer_encoded = label_encoder.fit_transform(y_train)
#             integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#             y_train = onehot_encoder.fit_transform(integer_encoded)    
            objective='multi:softprob'       
            num_class= 3
            max_depth=2
        elif file == 'covtype':
#             label_encoder = LabelEncoder()
#             onehot_encoder = OneHotEncoder(sparse=False)
#             integer_encoded = label_encoder.fit_transform(y_train)
#             integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#             y_train = onehot_encoder.fit_transform(integer_encoded)    
            objective='multi:softprob'       
            num_class= 7
            max_depth=2
        else:
            objective='binary:logistic'
            num_class=1
            max_depth=2
        learner = xgb.XGBRegressor(objective=objective, max_depth=max_depth, n_estimators=350, nthread=8,
                              eval_metric='logloss', eta=0, early_stopping_rounds=100, num_class=num_class)
        learner = learner.fit(X_train, y_train)  
        y_pred = learner.predict(X_test)    
    if  learner_model == 'XGB':
        if file in ['diabetic_data', 'covtype']:
            y_pred = np.asarray([np.argmax(line) for line in y_pred])
#             y_pred_inverse = label_encoder.inverse_transform([argmax(y_pred[0, :])])  
#             accuracy=metrics.accuracy_score(y_test, (y_pred_inverse > 0.5).astype(int))
#         else:
        accuracy=metrics.accuracy_score(y_test, (y_pred > 0.5).astype(int))
    else:
        accuracy=metrics.accuracy_score(y_test, y_pred)
    stop = timeit.default_timer()
    duration = stop - start
    print ("accuracy: {}".format(accuracy))
    print ("run time: {}".format(duration))
    return accuracy

    


## Load Data

In [8]:
####################################################################################################################
#################################################  DATA PREP ####################################################
####################################################################################################################
train_data = pd.read_csv(location + '/' + file + train_data)
data_test = pd.read_csv(location + '/' + file + test_data)
data_val = pd.read_csv(location + '/' + file + val_data)

In [9]:
train_data.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Batch_Index,quality
0,1,6.5,0.360,0.28,3.2,0.037,29.0,119.0,0.99080,3.25,0.65,12.4,23,1
1,1,6.1,0.320,0.24,1.5,0.036,38.0,124.0,0.98980,3.29,0.42,12.4,9,1
2,0,7.1,0.460,0.20,1.9,0.077,28.0,54.0,0.99560,3.37,0.64,10.4,1,1
3,0,8.4,0.745,0.11,1.9,0.090,16.0,63.0,0.99650,3.19,0.82,9.6,15,0
4,1,6.8,0.230,0.32,8.6,0.046,47.0,159.0,0.99452,3.08,0.52,10.5,12,1


In [10]:
data_val.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Batch_Index,quality
0,1,6.9,0.18,0.38,8.1,0.049,44.0,176.0,0.99580,3.30,0.54,9.8,22,1
1,0,7.8,0.56,0.19,2.1,0.081,15.0,105.0,0.99620,3.33,0.54,9.5,1,0
2,1,6.1,0.17,0.42,15.1,0.033,28.0,124.0,0.99684,2.87,0.47,9.5,3,0
3,1,6.8,0.17,0.74,2.4,0.053,61.0,182.0,0.99530,3.63,0.76,10.5,22,1
4,0,8.3,0.65,0.10,2.9,0.089,17.0,40.0,0.99803,3.29,0.55,9.5,15,0


In [11]:
data_test.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1,6.9,0.38,0.29,13.65,0.048,52.0,189.0,0.99784,3.00,0.60,9.5,1
1,1,6.7,0.32,0.32,1.70,0.031,31.0,114.0,0.98946,3.12,0.35,12.5,1
2,1,7.2,0.21,0.41,1.30,0.036,33.0,85.0,0.99200,3.17,0.51,10.4,0
3,1,8.5,0.16,0.33,1.00,0.076,17.0,57.0,0.99210,3.14,0.46,10.6,1
4,1,6.9,0.28,0.33,1.20,0.039,16.0,98.0,0.99040,3.07,0.39,11.7,1


## ROC

In [12]:
# binary encode


Results = []

X_test,y_test=data_separate(data_val,0)

del X_test['Batch_Index']

# X_test,y_test=data_separate(data_val,0)

print ("------------ALL------------------")
X_train,y_train=data_separate(train_data,1)

accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')

Results.append(['All',accuracy, 1-(len(train_data)/len(train_data))])

# print ("------------ENN------------------")
# X_train,y_train=data_separate(data_ENN,0)
    
# accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')
# Results.append(['ENN',accuracy,1-(len(data_ENN)/len(train_data))])

# print ("------------DROP3------------------")
# X_train,y_train=data_separate(data_DROP3,0)
# accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')

# Results.append(['DROP3',accuracy,1-(len(data_DROP3)/len(train_data))])

# print ("------------LDIS ------------------")
# X_train,y_train=data_separate(data_LDIS ,1)

# accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')

# Results.append(['LDIS',accuracy,1-(len(data_LDIS)/len(train_data))])


------------ALL------------------
accuracy: 0.8
run time: 0.5356102000000078


In [ ]:
Results

### Summarize 30 experiments into one dataframe

In [13]:
if file =='Wine':
#     A = pd.read_csv('Experiments_Wine_test_Q.csv', index_col=0) 
    file_name = 'Experiments/Wine/Experiments_Wine_Q'
if file == 'mushrooms':
    file_name = 'Experiments/mushrooms/Experiments_mushrooms_Q'
if file == 'adult':
    file_name = 'Experiments/adult/Experiments_adult_Q'
if file == 'diabetic_data':
    file_name = 'Experiments/diabetic_data/Experiments_diabetic_data_Q'
if file == 'covtype':
    file_name = 'Experiments/covtype/Experiments_covtype_Q'
        
# A = pd.read_csv(file_name + '.csv', index_col=0)

In [14]:
# Results_1 = pd.read_csv('results/temp_experiments_summary_' + file + '.csv')

In [15]:
# Results_1 = Results_1[~(Results_1.experiment==27)]
# del Results_1['Unnamed: 0']

In [16]:
col_list = ['experiment','decision_point', 'batches', 'validation_accuracy', 'data_reduction' ]
Results_1 = pd.DataFrame(columns = col_list )

for i in range(30):
# for i in range(30):
    # load relevant file 
    final_file_name = file_name + '_' + str(i) + '.csv' 
    print (final_file_name)
    A = pd.read_csv(final_file_name, index_col=0)
    
    
    # initial policy 
    s=A.shape[0]-1
    include = []
    exclude = []
    
    while len(include)<A.shape[0]:
        exclude.append(s)
        options = np.array(A.loc[s:s,])

        options[:, exclude] = -1
        a =  np.argmax(options)
        include.append(a)
        external_trashold = len(include)

        data_CIS = train_data[train_data.Batch_Index.isin(include)]
#         if len(data_CIS)!=0:
#             data_CIS = data_CIS.groupby('Batch_Index', group_keys=False).apply(pd.DataFrame.sample, frac=0.9)
#             data_CIS = data_CIS.sample(frac=0.85)                
        X_train,y_train=data_separate(data_CIS  ,1)
#         print(f"X_train shape {X_train.shape}")
        if len(X_train)==0:
            accuracy=0
        else:
            accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')
        size = 1-(len(data_CIS)/len(train_data))
        Results_1 = Results_1.append(pd.DataFrame([[i, external_trashold, list(include), round(accuracy,3),round(size,3)]], columns = col_list),ignore_index=True)
        s = a
        print(external_trashold)
        print (include)
        print (size)
    Results_1.drop(Results_1.tail(1).index,inplace=True)

Experiments/Wine/Experiments_Wine_Q_0.csv
accuracy: 0.6730769230769231
run time: 0.23325959999999668
1
[19]
0.9997861877271755
accuracy: 0.6442307692307693
run time: 0.38280590000000814
2
[19, 3]
0.9322215095146461
accuracy: 0.7557692307692307
run time: 0.37964569999999753
3
[19, 3, 20]
0.8973701090442592
accuracy: 0.7769230769230769
run time: 0.3901438999999982
4
[19, 3, 20, 2]
0.8541800299337182
accuracy: 0.7769230769230769
run time: 0.37956850000000486
5
[19, 3, 20, 2, 14]
0.8400684199273039
accuracy: 0.801923076923077
run time: 0.3905808000000093
6
[19, 3, 20, 2, 14, 7]
0.8007269617276032
accuracy: 0.7846153846153846
run time: 0.39794879999999466
7
[19, 3, 20, 2, 14, 7, 9]
0.7316655976053026
accuracy: 0.8096153846153846
run time: 0.4038613999999967
8
[19, 3, 20, 2, 14, 7, 9, 21]
0.700235193500107
accuracy: 0.801923076923077
run time: 0.41278710000000274
9
[19, 3, 20, 2, 14, 7, 9, 21, 5]
0.6741500962155228
accuracy: 0.7923076923076923
run time: 0.4255454000000043
10
[19, 3, 20, 2, 1

accuracy: 0.7923076923076923
run time: 0.47960860000000594
18
[19, 3, 2, 20, 15, 9, 21, 0, 8, 17, 1, 7, 5, 4, 16, 13, 12, 23]
0.11845199914475091
accuracy: 0.7846153846153846
run time: 0.4700677000000013
19
[19, 3, 2, 20, 15, 9, 21, 0, 8, 17, 1, 7, 5, 4, 16, 13, 12, 23, 10]
0.07462048321573655
accuracy: 0.7942307692307692
run time: 0.47756160000000136
20
[19, 3, 2, 20, 15, 9, 21, 0, 8, 17, 1, 7, 5, 4, 16, 13, 12, 23, 10, 22]
0.026298909557408545
accuracy: 0.7903846153846154
run time: 0.47008490000000336
21
[19, 3, 2, 20, 15, 9, 21, 0, 8, 17, 1, 7, 5, 4, 16, 13, 12, 23, 10, 22, 6]
0.025443660466110796
accuracy: 0.8
run time: 0.471382399999996
22
[19, 3, 2, 20, 15, 9, 21, 0, 8, 17, 1, 7, 5, 4, 16, 13, 12, 23, 10, 22, 6, 14]
0.01133205045969643
accuracy: 0.7942307692307692
run time: 0.4694722999999925
23
[19, 3, 2, 20, 15, 9, 21, 0, 8, 17, 1, 7, 5, 4, 16, 13, 12, 23, 10, 22, 6, 14, 18]
0.0036348086380157962
accuracy: 0.8
run time: 0.4718819999999937
24
[19, 3, 2, 20, 15, 9, 21, 0, 8, 17, 

accuracy: 0.7788461538461539
run time: 0.4120632999999998
9
[19, 20, 2, 1, 3, 8, 15, 7, 9]
0.5740859525336754
accuracy: 0.7769230769230769
run time: 0.41379309999999236
10
[19, 20, 2, 1, 3, 8, 15, 7, 9, 4]
0.5477870429762668
accuracy: 0.7846153846153846
run time: 0.42413270000000125
11
[19, 20, 2, 1, 3, 8, 15, 7, 9, 4, 0]
0.47658755612572157
accuracy: 0.7865384615384615
run time: 0.431272000000007
12
[19, 20, 2, 1, 3, 8, 15, 7, 9, 4, 0, 16]
0.4197134915544152
accuracy: 0.7807692307692308
run time: 0.4485018000000025
13
[19, 20, 2, 1, 3, 8, 15, 7, 9, 4, 0, 16, 21]
0.3882830874492196
accuracy: 0.7846153846153846
run time: 0.4390773000000081
14
[19, 20, 2, 1, 3, 8, 15, 7, 9, 4, 0, 16, 21, 22]
0.33996151379089157
accuracy: 0.7884615384615384
run time: 0.46020129999999426
15
[19, 20, 2, 1, 3, 8, 15, 7, 9, 4, 0, 16, 21, 22, 13]
0.2800940774000428
accuracy: 0.7942307692307692
run time: 0.44295660000000225
16
[19, 20, 2, 1, 3, 8, 15, 7, 9, 4, 0, 16, 21, 22, 13, 12]
0.2133846482788112
accuracy:

accuracy: 0.8
run time: 0.4509306999999865
24
[19, 3, 8, 0, 2, 20, 1, 15, 13, 4, 7, 9, 21, 14, 12, 5, 16, 23, 17, 18, 6, 22, 10, 11]
0.0
accuracy: 0.8
run time: 0.453187000000014
25
[19, 3, 8, 0, 2, 20, 1, 15, 13, 4, 7, 9, 21, 14, 12, 5, 16, 23, 17, 18, 6, 22, 10, 11, 0]
0.0
Experiments/Wine/Experiments_Wine_Q_8.csv
accuracy: 0.6730769230769231
run time: 0.19412929999998596
1
[19]
0.9997861877271755
accuracy: 0.6442307692307693
run time: 0.38149219999999673
2
[19, 3]
0.9322215095146461
accuracy: 0.7692307692307693
run time: 0.37863740000000234
3
[19, 3, 9]
0.8631601453923455
accuracy: 0.7788461538461539
run time: 0.3741579999999942
4
[19, 3, 9, 20]
0.8283087449219585
accuracy: 0.7692307692307693
run time: 0.4566164000000015
5
[19, 3, 9, 20, 22]
0.7799871712636306
accuracy: 0.7615384615384615
run time: 0.41005769999998165
6
[19, 3, 9, 20, 22, 2]
0.7367970921530895
accuracy: 0.7826923076923077
run time: 0.3969695000000115
7
[19, 3, 9, 20, 22, 2, 18]
0.729099850331409
accuracy: 0.77307692

accuracy: 0.7942307692307692
run time: 0.446507199999985
16
[19, 9, 20, 2, 3, 15, 21, 7, 0, 18, 22, 1, 8, 12, 17, 5]
0.2501603592046183
accuracy: 0.7865384615384615
run time: 0.4514092000000005
17
[19, 9, 20, 2, 3, 15, 21, 7, 0, 18, 22, 1, 8, 12, 17, 5, 4]
0.22386144964720978
accuracy: 0.8076923076923077
run time: 0.45676090000000613
18
[19, 9, 20, 2, 3, 15, 21, 7, 0, 18, 22, 1, 8, 12, 17, 5, 4, 13]
0.16399401325636087
accuracy: 0.8096153846153846
run time: 0.4596284000000139
19
[19, 9, 20, 2, 3, 15, 21, 7, 0, 18, 22, 1, 8, 12, 17, 5, 4, 13, 14]
0.1498824032499465
accuracy: 0.8115384615384615
run time: 0.46249579999999924
20
[19, 9, 20, 2, 3, 15, 21, 7, 0, 18, 22, 1, 8, 12, 17, 5, 4, 13, 14, 11]
0.1462475946119307
accuracy: 0.7923076923076923
run time: 0.46203819999999496
21
[19, 9, 20, 2, 3, 15, 21, 7, 0, 18, 22, 1, 8, 12, 17, 5, 4, 13, 14, 11, 16]
0.08937353004062432
accuracy: 0.7980769230769231
run time: 0.46548160000000394
22
[19, 9, 20, 2, 3, 15, 21, 7, 0, 18, 22, 1, 8, 12, 17, 5,

...

accuracy: 0.7807692307692308
run time: 0.4674851999999987
14
[19, 23, 0, 20, 2, 7, 3, 15, 1, 9, 22, 13, 8, 12]
0.28330126149240964
accuracy: 0.7942307692307692
run time: 0.46463970000002064
15
[19, 23, 0, 20, 2, 7, 3, 15, 1, 9, 22, 13, 8, 12, 21]
0.25187085738721404
accuracy: 0.7884615384615384
run time: 0.4831704000000059
16
[19, 23, 0, 20, 2, 7, 3, 15, 1, 9, 22, 13, 8, 12, 21, 17]
0.17938849689972203
accuracy: 0.7865384615384615
run time: 0.4551700000000096
17
[19, 23, 0, 20, 2, 7, 3, 15, 1, 9, 22, 13, 8, 12, 21, 17, 18]
0.1716912550780415
accuracy: 0.7884615384615384
run time: 0.4748824000000127
18
[19, 23, 0, 20, 2, 7, 3, 15, 1, 9, 22, 13, 8, 12, 21, 17, 18, 16]
0.11481719050673511
accuracy: 0.7980769230769231
run time: 0.4728429999999548
19
[19, 23, 0, 20, 2, 7, 3, 15, 1, 9, 22, 13, 8, 12, 21, 17, 18, 16, 5]
0.08873209322215092
accuracy: 0.8
run time: 0.47733030000000554
20
[19, 23, 0, 20, 2, 7, 3, 15, 1, 9, 22, 13, 8, 12, 21, 17, 18, 16, 5, 14]
0.07462048321573655
accuracy: 0.796

accuracy: 0.7269230769230769
run time: 0.3818282000000295
4
[19, 9, 14, 2]
0.8734231344879196
accuracy: 0.7480769230769231
run time: 0.38494300000002113
5
[19, 9, 14, 2, 20]
0.8385717340175326
accuracy: 0.7826923076923077
run time: 0.39131190000000515
6
[19, 9, 14, 2, 20, 3]
0.7710070558050032
accuracy: 0.7865384615384615
run time: 0.3937870999999973
7
[19, 9, 14, 2, 20, 3, 15]
0.7036561898652982
accuracy: 0.7961538461538461
run time: 0.4191315000000486
8
[19, 9, 14, 2, 20, 3, 15, 13]
0.6437887534744494
accuracy: 0.8153846153846154
run time: 0.4134305999999697
9
[19, 9, 14, 2, 20, 3, 15, 13, 0]
0.5725892666239042
accuracy: 0.8173076923076923
run time: 0.4283971999999494
10
[19, 9, 14, 2, 20, 3, 15, 13, 0, 16]
0.5157152020525978
accuracy: 0.7980769230769231
run time: 0.446521099999984
11
[19, 9, 14, 2, 20, 3, 15, 13, 0, 16, 8]
0.4733803720333547
accuracy: 0.8038461538461539
run time: 0.44391780000000836
12
[19, 9, 14, 2, 20, 3, 15, 13, 0, 16, 8, 1]
0.41137481291426126
accuracy: 0.798076

accuracy: 0.8134615384615385
run time: 0.47894200000001774
20
[19, 3, 20, 2, 11, 9, 21, 15, 18, 5, 0, 8, 22, 4, 14, 12, 17, 13, 7, 23]
0.16356638871071194
accuracy: 0.801923076923077
run time: 0.5393861000000015
21
[19, 3, 20, 2, 11, 9, 21, 15, 18, 5, 0, 8, 22, 4, 14, 12, 17, 13, 7, 23, 1]
0.1015608295916186
accuracy: 0.7980769230769231
run time: 0.4826631999999904
22
[19, 3, 20, 2, 11, 9, 21, 15, 18, 5, 0, 8, 22, 4, 14, 12, 17, 13, 7, 23, 1, 16]
0.044686765020312214
accuracy: 0.8038461538461539
run time: 0.485707600000012
23
[19, 3, 20, 2, 11, 9, 21, 15, 18, 5, 0, 8, 22, 4, 14, 12, 17, 13, 7, 23, 1, 16, 6]
0.04383151592901435
accuracy: 0.8
run time: 0.48759669999998323
24
[19, 3, 20, 2, 11, 9, 21, 15, 18, 5, 0, 8, 22, 4, 14, 12, 17, 13, 7, 23, 1, 16, 6, 10]
0.0
accuracy: 0.8
run time: 0.472664199999997
25
[19, 3, 20, 2, 11, 9, 21, 15, 18, 5, 0, 8, 22, 4, 14, 12, 17, 13, 7, 23, 1, 16, 6, 10, 0]
0.0
Experiments/Wine/Experiments_Wine_Q_24.csv
accuracy: 0.6730769230769231
run time: 0.2004

accuracy: 0.7961538461538461
run time: 0.42067990000003874
12
[19, 3, 9, 20, 2, 18, 7, 5, 13, 4, 15, 21]
0.5270472525122942
accuracy: 0.801923076923077
run time: 0.4317060000000197
13
[19, 3, 9, 20, 2, 18, 7, 5, 13, 4, 15, 21, 12]
0.46033782339106266
accuracy: 0.7980769230769231
run time: 0.41945970000000443
14
[19, 3, 9, 20, 2, 18, 7, 5, 13, 4, 15, 21, 12, 22]
0.41201624973273465
accuracy: 0.8096153846153846
run time: 0.42403450000000475
15
[19, 3, 9, 20, 2, 18, 7, 5, 13, 4, 15, 21, 12, 22, 0]
0.34081676288218943
accuracy: 0.7903846153846154
run time: 0.44349429999999757
16
[19, 3, 9, 20, 2, 18, 7, 5, 13, 4, 15, 21, 12, 22, 0, 23]
0.2961299978618773
accuracy: 0.7980769230769231
run time: 0.44713930000000346
17
[19, 3, 9, 20, 2, 18, 7, 5, 13, 4, 15, 21, 12, 22, 0, 23, 16]
0.23925593329057093
accuracy: 0.7865384615384615
run time: 0.4552390999999716
18
[19, 3, 9, 20, 2, 18, 7, 5, 13, 4, 15, 21, 12, 22, 0, 23, 16, 1]
0.1772503741714775
accuracy: 0.7903846153846154
run time: 0.45315329999

accuracy: 0.6730769230769231
run time: 0.201681600000029
1
[19]
0.9997861877271755
accuracy: 0.6384615384615384
run time: 0.37287079999998696
2
[19, 9]
0.930724823604875
accuracy: 0.6865384615384615
run time: 0.3785184000000186
3
[19, 9, 2]
0.887534744494334
accuracy: 0.7480769230769231
run time: 0.3828866999999718
4
[19, 9, 2, 20]
0.852683344023947
accuracy: 0.7673076923076924
run time: 0.3928650999999945
5
[19, 9, 2, 20, 3]
0.7851186658114175
accuracy: 0.7653846153846153
run time: 0.3964620000000423
6
[19, 9, 2, 20, 3, 8]
0.7427838357921746
accuracy: 0.7461538461538462
run time: 0.41194180000002234
7
[19, 9, 2, 20, 3, 8, 0]
0.6715843489416293
accuracy: 0.7557692307692307
run time: 0.4173079000000257
8
[19, 9, 2, 20, 3, 8, 0, 1]
0.6095787898225358
accuracy: 0.7711538461538462
run time: 0.4089533000000074
9
[19, 9, 2, 20, 3, 8, 0, 1, 22]
0.5612572161642078
accuracy: 0.7788461538461539
run time: 0.4192706000000044
10
[19, 9, 2, 20, 3, 8, 0, 1, 22, 15]
0.4939063502245029
accuracy: 0.7807

In [17]:
# Results_1[Results_1.decision_point==3]

In [18]:
# Results_1

In [19]:
Results_1['sorted_batches'] = Results_1['batches'].apply(sorted)
# .apply(lambda x: x.strip("[]").split(", ")).apply(sorted)
Results_1['sorted_batches'] = Results_1['sorted_batches'].apply(str)
Results_1['batches'] = Results_1['batches'].apply(str)

In [20]:
# Results_1.to_csv('results/experiments_summary_' + file + '.csv')
# Results_1 = pd.read_csv('results/experiments_summary_' + file + '.csv')

In [21]:
Results_1.shape

(720, 6)

In [22]:
df1 = Results_1[['decision_point','validation_accuracy']].groupby('decision_point').validation_accuracy.mean().reset_index(name='validation_accuracy_mean')
df1['validation_accuracy_std'] = Results_1[['decision_point','validation_accuracy']].groupby('decision_point').validation_accuracy.transform(np.std)
df1['data_reduction_mean'] = Results_1[['decision_point','data_reduction']].groupby('decision_point').data_reduction.transform(np.mean)
df1['data_reduction_std'] = Results_1[['decision_point','data_reduction']].groupby('decision_point').data_reduction.transform(np.std)
df1['validation_accuracy_best_policy'] = Results_1[['decision_point','validation_accuracy']].groupby('decision_point').validation_accuracy.transform(max)
df1['data_reduction_best_policy'] = 0
df1['best_policy'] = '[]'

In [23]:
temp = Results_1[['decision_point','validation_accuracy','data_reduction','sorted_batches']].drop_duplicates()
temp = temp.rename(columns={'validation_accuracy':"best_validation_accuracy","data_reduction":"best_data_reduction", "sorted_batches":"best_sorted_batches"})
for i in df1.index:
    decision_point = df1['decision_point'].iloc[i]
    validation_accuracy = df1['validation_accuracy_best_policy'].iloc[i]
    df1['data_reduction_best_policy'].iloc[i] = temp[(temp.decision_point==decision_point)&(temp.best_validation_accuracy==validation_accuracy)].best_data_reduction.values[0]
    df1['best_policy'].iloc[i] = temp[(temp.decision_point==decision_point)&(temp.best_validation_accuracy==validation_accuracy)].best_sorted_batches.values[0]

C:\Users\mmoran1\Anaconda3\envs\base_tensorflow\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [24]:
df1

,decision_point,validation_accuracy_mean,validation_accuracy_std,data_reduction_mean,data_reduction_std,validation_accuracy_best_policy,data_reduction_best_policy,best_policy
0,1,0.673000,0.000000,1.000000,0.000000,0.673,1.000,[19]
1,2,0.644833,0.046998,0.939033,0.012366,0.713,0.957,"[2, 19]"
2,3,0.732633,0.045537,0.894733,0.018243,0.771,0.889,"[2, 3, 19]"
3,4,0.761000,0.020665,0.846700,0.013251,0.779,0.828,"[3, 9, 19, 20]"
4,5,0.761867,0.014371,0.795600,0.020771,0.796,0.794,"[2, 3, 13, 19, 20]"
5,6,0.769733,0.014242,0.746867,0.024427,0.802,0.801,"[2, 3, 7, 14, 19, 20]"
6,7,0.773733,0.016062,0.696533,0.027198,0.796,0.669,"[2, 3, 9, 15, 19, 20, 22]"
7,8,0.780633,0.016232,0.646167,0.037531,0.813,0.632,"[0, 2, 3, 7, 15, 19, 20, 23]"
8,9,0.783400,0.014792,0.594667,0.039201,0.815,0.573,"[0, 2, 3, 9, 13, 14, 15, 19, 20]"
9,10,0.784633,0.011790,0.550200,0.044268,0.817,0.516,"[0, 2, 3, 9, 13, 14, 15, 16, 19, 20]"


In [25]:
# sorted_batches_cnt = Results_1.groupby(['decision_point','sorted_batches','validation_accuracy','data_reduction']).sorted_batches.count().reset_index(name='cnt')
# sorted_batches_cnt['most_frequent'] = sorted_batches_cnt[['decision_point','sorted_batches','cnt']].groupby('decision_point').cnt.transform(max)
# sorted_batches_cnt = sorted_batches_cnt[sorted_batches_cnt.cnt == sorted_batches_cnt.most_frequent]
# sorted_batches_cnt['best_accurcy'] = sorted_batches_cnt[['decision_point','sorted_batches','validation_accuracy']].groupby('decision_point').validation_accuracy.transform(max)
# sorted_batches_cnt = sorted_batches_cnt[sorted_batches_cnt.validation_accuracy == sorted_batches_cnt.best_accurcy]
# sorted_batches_cnt['best_data_reduction'] = sorted_batches_cnt[['decision_point','sorted_batches','data_reduction']].groupby('decision_point').data_reduction.transform(max)
# sorted_batches_cnt = sorted_batches_cnt[sorted_batches_cnt.data_reduction == sorted_batches_cnt.best_data_reduction]
# most_frequent = sorted_batches_cnt[['decision_point','sorted_batches','validation_accuracy','data_reduction','most_frequent']].rename(columns = {'sorted_batches':'sorted_batches_most_frequent','validation_accuracy': 'validation_accuracy_most_frequent','data_reduction':'data_reduction_most_frequent'}).reset_index(drop=True)
# del most_frequent['decision_point']

In [26]:
# results["best_policy"].apply(lambda x: list(map(int, x)))

In [28]:
# results = pd.concat([df1, most_frequent], axis=1)

# results.head()

In [31]:
results =  df1.copy()

In [72]:
# results.to_csv('results/results_' + file + '.csv')
# results = pd.read_csv('results/results_' + file + '.csv')

In [ ]:
# results.to_csv('results/' + file + '_train_validation_summary.csv')
# results.to_excel('results/' + file + '_train_validation_summary.xlsx')

In [32]:
max_accuracy_validation = results.validation_accuracy_best_policy.max()

In [30]:
# results = pd.read_csv('results/' + file + '_train_validation_summary.csv')

In [ ]:
results

In [ ]:
include

### create sampling for CIS data

In [33]:
include_results = results.copy()
# results[results.decision_point.isin(relevant_decision_point_best)]

print(f"X_test shape {X_test.shape}")
sampling_CIS = []
import ast

for index, include in include_results.best_policy.items():
#     include = [include.strip("'[]'")] 
#     include = [int(i) for i in include] 
    frac = 1
    while frac > 0.4 :
        print(include)
        print(frac)
        data_CIS = train_data[train_data.Batch_Index.isin(ast.literal_eval(include))]
#         data_CIS = train_data[train_data.Batch_Index.isin(include)]
        data_CIS = data_CIS.groupby('Batch_Index', group_keys=False).apply(pd.DataFrame.sample, frac=frac)
        X_train,y_train=data_separate(data_CIS ,1)
        print(f"X_train shape {X_train.shape}")
        if len(X_train)==0:
            accuracy=0
        else:
            accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')
        size = 1-(len(data_CIS)/len(train_data))
        effeicency = round((accuracy/max_accuracy_validation) * (size), 3)
        sampling_CIS.append([index + 1, str(include), "CIS Batches = " + str(include), accuracy,size, effeicency, frac])
        frac = round(frac - 0.1,2)
        
    

X_test shape (520, 12)
[19]
1
X_train shape (1, 12)
accuracy: 0.6730769230769231
run time: 0.2060477999999648
[19]
0.9
X_train shape (1, 12)
accuracy: 0.6730769230769231
run time: 0.196921800000041
[19]
0.8
X_train shape (1, 12)
accuracy: 0.6730769230769231
run time: 0.19852670000000217
[19]
0.7
X_train shape (1, 12)
accuracy: 0.6730769230769231
run time: 0.19790469999998095
[19]
0.6
X_train shape (1, 12)
accuracy: 0.6730769230769231
run time: 0.20041600000001836
[19]
0.5
X_train shape (0, 12)
[2, 19]
1
X_train shape (203, 12)
accuracy: 0.7134615384615385
run time: 0.37322119999998904
[2, 19]
0.9
X_train shape (183, 12)
accuracy: 0.7076923076923077
run time: 0.3707694999999944
[2, 19]
0.8
X_train shape (163, 12)
accuracy: 0.7269230769230769
run time: 0.35728210000002036
[2, 19]
0.7
X_train shape (142, 12)
accuracy: 0.7057692307692308
run time: 0.36381389999996827
[2, 19]
0.6
X_train shape (122, 12)
accuracy: 0.7019230769230769
run time: 0.3506398000000104
[2, 19]
0.5
X_train shape (101

accuracy: 0.7846153846153846
run time: 0.41823640000001205
[0, 1, 2, 3, 5, 7, 8, 9, 13, 14, 15, 19, 20]
0.6
X_train shape (1677, 12)
accuracy: 0.7903846153846154
run time: 0.41571279999999433
[0, 1, 2, 3, 5, 7, 8, 9, 13, 14, 15, 19, 20]
0.5
X_train shape (1397, 12)
accuracy: 0.7692307692307693
run time: 0.4049135999999862
[0, 1, 2, 3, 7, 8, 9, 13, 14, 15, 16, 18, 19, 20]
1
X_train shape (2973, 12)
accuracy: 0.8096153846153846
run time: 0.4471292000000062
[0, 1, 2, 3, 7, 8, 9, 13, 14, 15, 16, 18, 19, 20]
0.9
X_train shape (2676, 12)
accuracy: 0.8
run time: 0.438782099999969
[0, 1, 2, 3, 7, 8, 9, 13, 14, 15, 16, 18, 19, 20]
0.8
X_train shape (2378, 12)
accuracy: 0.7884615384615384
run time: 0.42085170000001426
[0, 1, 2, 3, 7, 8, 9, 13, 14, 15, 16, 18, 19, 20]
0.7
X_train shape (2080, 12)
accuracy: 0.8038461538461539
run time: 0.41732240000004595
[0, 1, 2, 3, 7, 8, 9, 13, 14, 15, 16, 18, 19, 20]
0.6
X_train shape (1786, 12)
accuracy: 0.7807692307692308
run time: 0.40494569999998475
[0, 1,

accuracy: 0.8
run time: 0.42458539999995537
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
1
X_train shape (4472, 12)
accuracy: 0.8038461538461539
run time: 0.47217799999998533
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
0.9
X_train shape (4025, 12)
accuracy: 0.7942307692307692
run time: 0.4603078000000096
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
0.8
X_train shape (3578, 12)
accuracy: 0.7884615384615384
run time: 0.44685630000003584
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
0.7
X_train shape (3128, 12)
accuracy: 0.801923076923077
run time: 0.4471036999999569
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
0.6
X_train shape (2684, 12)
accuracy: 0.7865384615384615
run time: 0.43777609999995093
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
0.5
X_train shape (2237

In [34]:
sampling_CIS_table = pd.DataFrame(sampling_CIS, columns =['decision_point', 'policy', 'CIS_Batches', 'XGboost_Model_Accuracy', 'Data_Reduction' , 'effeicency', 'frac'])
# sampling_CIS_table

In [35]:
sampling_CIS_table[sampling_CIS_table.decision_point==3]

,decision_point,policy,CIS_Batches,XGboost_Model_Accuracy,Data_Reduction,effeicency,frac
12,3,"[2, 3, 19]","CIS Batches = [2, 3, 19]",0.771154,0.889031,0.835,1.0
13,3,"[2, 3, 19]","CIS Batches = [2, 3, 19]",0.751923,0.900150,0.824,0.9
14,3,"[2, 3, 19]","CIS Batches = [2, 3, 19]",0.782692,0.911054,0.869,0.8
15,3,"[2, 3, 19]","CIS Batches = [2, 3, 19]",0.748077,0.922386,0.840,0.7
16,3,"[2, 3, 19]","CIS Batches = [2, 3, 19]",0.763462,0.933291,0.868,0.6
17,3,"[2, 3, 19]","CIS Batches = [2, 3, 19]",0.734615,0.944623,0.845,0.5


In [62]:
sampling_CIS_table['max_effeicency'] = sampling_CIS_table.groupby(['CIS_Batches'])['effeicency'].transform(max)
sampling_CIS_table = sampling_CIS_table[sampling_CIS_table['max_effeicency']== sampling_CIS_table['effeicency']]
sampling_CIS_table['min_frac'] = sampling_CIS_table.groupby(['CIS_Batches'])['frac'].transform(min)
sampling_CIS_table = sampling_CIS_table[sampling_CIS_table['min_frac']== sampling_CIS_table['frac']]
sampling_CIS_table.reset_index(drop=True)


C:\Users\mmoran1\Anaconda3\envs\base_tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,decision_point,policy,CIS_Batches,XGboost_Model_Accuracy,Data_Reduction,effeicency,frac,max_effeicency,min_frac
4,1,[19],CIS Batches = [19],0.673077,0.999786,0.829,0.6,0.829,0.6
9,2,"[13, 19]","CIS Batches = [13, 19]",0.688462,0.957879,0.812,0.7,0.812,0.7
14,3,"[13, 19, 20]","CIS Batches = [13, 19, 20]",0.782692,0.924097,0.891,0.8,0.891,0.8
20,4,"[3, 13, 19, 20]","CIS Batches = [3, 13, 19, 20]",0.775000,0.870002,0.830,0.8,0.830,0.8
29,5,"[2, 3, 13, 19, 20]","CIS Batches = [2, 3, 13, 19, 20]",0.776923,0.897156,0.858,0.5,0.858,0.5
35,6,"[2, 3, 8, 13, 19, 20]","CIS Batches = [2, 3, 8, 13, 19, 20]",0.778846,0.875989,0.840,0.5,0.840,0.5
41,7,"[2, 3, 8, 13, 15, 19, 20]","CIS Batches = [2, 3, 8, 13, 15, 19, 20]",0.784615,0.842207,0.814,0.5,0.814,0.5
47,8,"[1, 2, 3, 8, 13, 15, 19, 20]","CIS Batches = [1, 2, 3, 8, 13, 15, 19, 20]",0.763462,0.811204,0.763,0.5,0.763,0.5
53,9,"[0, 1, 2, 3, 9, 15, 17, 19, 20]","CIS Batches = [0, 1, 2, 3, 9, 15, 17, 19, 20]",0.794231,0.755826,0.739,0.5,0.739,0.5
59,10,"[0, 1, 2, 3, 8, 9, 15, 17, 19, 20]","CIS Batches = [0, 1, 2, 3, 8, 9, 15, 17, 19, 20]",0.776923,0.734659,0.703,0.5,0.703,0.5


In [ ]:
sampling_CIS_table[(sampling_CIS_table.decision_point==3)]

In [63]:
# sampling_CIS_table.to_csv('results/' + file + '_train_validation_sampling_CIS_table.csv')
# sampling_CIS_table[["decision_point","policy"]].rename(columns={"decision_point":"Decision Point","policy":"CIS Batches"}).to_csv('results/' + file + 'decision_point.csv')

In [64]:
# sampling_CIS_table= pd.read_csv('results/' + file + '_train_validation_sampling_CIS_table.csv')

In [65]:
best_results = results[['decision_point','best_policy']]
best_results = best_results.rename(columns={'best_policy':'policy'})
best_results['frac'] = 1
best_results['XGboost_Model_Accuracy'] = results['validation_accuracy_best_policy']
best_results['Data_Reduction'] = results['data_reduction_best_policy']
best_results['effeicency'] = (results['validation_accuracy_best_policy']/max_accuracy_validation) * results['data_reduction_best_policy']
best_results.head()

,decision_point,policy,frac,XGboost_Model_Accuracy,Data_Reduction,effeicency
0,1,[19],1,0.673,1.000,0.828818
1,2,"[13, 19]",1,0.675,0.940,0.781404
2,3,"[13, 19, 20]",1,0.763,0.905,0.850388
3,4,"[3, 13, 19, 20]",1,0.775,0.838,0.799815
4,5,"[2, 3, 13, 19, 20]",1,0.796,0.794,0.778355


In [66]:
best_results = best_results.append(sampling_CIS_table[['decision_point', 'policy', 'frac','XGboost_Model_Accuracy','Data_Reduction','effeicency']])
best_results = best_results.sort_values(by='Data_Reduction', ascending=False)
best_results['name'] = best_results['decision_point'].astype(str) + ' frac=' + best_results['frac'].astype(str)

best_results = best_results.reset_index(drop=True)
best_results['index1'] = best_results.index + 1

In [67]:
best_results

,decision_point,policy,frac,XGboost_Model_Accuracy,Data_Reduction,effeicency,name,index1
0,1,[19],1.0,0.673000,1.000000,0.828818,1 frac=1.0,1
1,1,[19],0.6,0.673077,0.999786,0.829000,1 frac=0.6,2
2,2,"[13, 19]",0.7,0.688462,0.957879,0.812000,2 frac=0.7,3
3,2,"[13, 19]",1.0,0.675000,0.940000,0.781404,2 frac=1.0,4
4,3,"[13, 19, 20]",0.8,0.782692,0.924097,0.891000,3 frac=0.8,5
5,3,"[13, 19, 20]",1.0,0.763000,0.905000,0.850388,3 frac=1.0,6
6,5,"[2, 3, 13, 19, 20]",0.5,0.776923,0.897156,0.858000,5 frac=0.5,7
7,6,"[2, 3, 8, 13, 19, 20]",0.5,0.778846,0.875989,0.840000,6 frac=0.5,8
8,4,"[3, 13, 19, 20]",0.8,0.775000,0.870002,0.830000,4 frac=0.8,9
9,7,"[2, 3, 8, 13, 15, 19, 20]",0.5,0.784615,0.842207,0.814000,7 frac=0.5,10


In [23]:
# best_results['index1'] = best_results.index + 1

In [21]:
# results = results.loc[0:85]

### pareto font 

#### before and after point check

In [36]:
def below_line(x1, y1, x2, y2, x, y):
    d = ((x - x1)*(y2 - y1)) - ((y - y1)*(x2 -x1))
    m = (y2 - y1)/(x2 - x1)
    return d

In [37]:

relevant_decision_point_best = []
acc_col = 'validation_accuracy_best_policy'
size_col = 'data_reduction_best_policy'

point1 = 1
for i in results.decision_point.values:
    point2 = min(i + 1, results.decision_point.max())
    if i == 1:
        relevant_decision_point_best.append(i) 
    else:
        y1 = results[acc_col][results.decision_point == point1].values[0]
        x1 = results[size_col][results.decision_point == point1].values[0]
        y2 = results[acc_col][results.decision_point == point2].values[0]
        x2 = results[size_col][results.decision_point == point2].values[0]
        y = results[acc_col][results.decision_point == i].values[0]
        x = results[size_col][results.decision_point == i].values[0]
        d = below_line(x1, y1, x2, y2, x, y)
        if d >=0:
            relevant_decision_point_best.append(i)
            point1 = i
            
relevant_decision_point_sample = []

acc_col = 'XGboost_Model_Accuracy'
size_col = 'Data_Reduction'

point1 = 1
for i in sampling_CIS_table.decision_point.values:
    point2 = min(i + 1, sampling_CIS_table.decision_point.max())
    if i == 1:
        relevant_decision_point_sample.append(i) 
    else:
        y1 = sampling_CIS_table[acc_col][sampling_CIS_table.decision_point == point1].values[0]
        x1 = sampling_CIS_table[size_col][sampling_CIS_table.decision_point == point1].values[0]
        y2 = sampling_CIS_table[acc_col][sampling_CIS_table.decision_point == point2].values[0]
        x2 = sampling_CIS_table[size_col][sampling_CIS_table.decision_point == point2].values[0]
        y = sampling_CIS_table[acc_col][sampling_CIS_table.decision_point == i].values[0]
        x = sampling_CIS_table[size_col][sampling_CIS_table.decision_point == i].values[0]
        d = below_line(x1, y1, x2, y2, x, y)
        if d >=0:
            relevant_decision_point_sample.append(i)
            point1 = i
            
relevant_decision_point_all = []

acc_col = 'XGboost_Model_Accuracy'
size_col = 'Data_Reduction'

point1 = 1
for i in best_results.index1.values:
    point2 = min(i+1, best_results.index1.max())
    if i == 1:
        relevant_decision_point_all.append(i) 
    else:
        y1 = best_results[acc_col][best_results.index1 == point1].values[0]
        x1 = best_results[size_col][best_results.index1 == point1].values[0]
        y2 = best_results[acc_col][best_results.index1 == point2].values[0]
        x2 = best_results[size_col][best_results.index1 == point2].values[0]
        y = best_results[acc_col][best_results.index1 == i].values[0]
        x = best_results[size_col][best_results.index1 == i].values[0]
        d = below_line(x1, y1, x2, y2, x, y)
        if d >=0:
            relevant_decision_point_all.append(i)
            point1 = i

            
relevant_decision_point_all_before =  relevant_decision_point_all.copy()
stop =0 
while stop ==0:
    relevant_decision_point_all_final = []
    point1 = 1
    for i, val in enumerate(relevant_decision_point_all_before): 
        if i == len(relevant_decision_point_all_before) - 1:
            point2 = relevant_decision_point_all_before[-1]
        else:
            point2 = relevant_decision_point_all_before[i+1]
        if val == 1:
            relevant_decision_point_all_final.append(val) 
        else:
            y1 = best_results[acc_col][best_results.index1 == point1].values[0]
            x1 = best_results[size_col][best_results.index1 == point1].values[0]
            y2 = best_results[acc_col][best_results.index1 == point2].values[0]
            x2 = best_results[size_col][best_results.index1 == point2].values[0]
            y = best_results[acc_col][best_results.index1 == val].values[0]
            x = best_results[size_col][best_results.index1 == val].values[0]
            d = below_line(x1, y1, x2, y2, x, y)
            if d >=0:
                relevant_decision_point_all_final.append(val)
                point1 = val
    if relevant_decision_point_all_before == relevant_decision_point_all_final:
        stop = 1
    else:
        relevant_decision_point_all_before = relevant_decision_point_all_final.copy()
    
    
    
print(relevant_decision_point_best)
print(relevant_decision_point_sample)
print(relevant_decision_point_all)
print(relevant_decision_point_all_final)

[1, 3, 5, 7, 12, 15, 16, 18, 19, 21, 23, 24]
[1, 3, 5, 8, 12, 13, 14, 15, 18, 20, 23, 24]
[1, 4, 8, 13, 16, 18, 19, 21, 26, 28, 31, 38, 40, 42, 43, 45, 47, 48]
[1, 4, 18, 26, 38, 42, 43, 47, 48]


In [ ]:
 'Wine': [[1, 3, 7, 13, 14, 17, 22, 24, 26, 30, 31, 33, 36, 43, 44, 47, 48],
  [1, 3, 7, 13, 24, 36, 43, 47, 48]],

In [ ]:
# Adult
# relevant_decision_point_all_final = relevant_decision_point_all_final[0:6] + relevant_decision_point_all_final[14:]
# covtype
#relevant_decision_point_all_final =  relevant_decision_point_all_final[0:14] + relevant_decision_point_all_final[-3:]

In [ ]:
pareto = best_results[best_results.index1.isin(relevant_decision_point_all_final)]
# pareto.to_csv('results/' + file + '_train_validation_pareto_font_results.csv')

In [24]:
# pareto= pd.read_csv('results/' + file + '_train_validation_pareto_font_results.csv')

In [ ]:

# def identify_pareto(scores):
#     # Count number of items
#     population_size = scores.shape[0]
#     # Create a NumPy index for scores on the pareto front (zero indexed)
#     population_ids = np.arange(population_size)
#     # Create a starting list of items on the Pareto front
#     # All items start off as being labelled as on the Parteo front
#     pareto_front = np.ones(population_size, dtype=bool)
#     # Loop through each item. This will then be compared with all other items
#     for i in range(population_size):
#         # Loop through all other items
#         for j in range(population_size):
#             # Check if our 'i' pint is dominated by out 'j' point
#             if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
#                 # j dominates i. Label 'i' point as not on Pareto front
#                 pareto_front[i] = 0
#                 # Stop further comparisons with 'i' (no more comparisons needed)
#                 break
#     # Return ids of scenarios on pareto front
#     return population_ids[pareto_front]



# acc_col = 'XGboost_Model_Accuracy'
# size_col = 'Data_Reduction'
# scores = np.array([ np.array(x) for x in best_results[[acc_col, size_col]].values ])
# pareto = identify_pareto(scores) +1
# relevant_decision_point_all = list(pareto)
# relevant_decision_point_all

In [ ]:
results

### Plot results

In [26]:
import seaborn as sns

%matplotlib notebook


## Baeline Algorithm ## 
key, y, x = zip(*Results)
df = pd.DataFrame({'key':key , 'x':x,  'y':y})


## CIS error plot ## 

df_mean = results[['decision_point','validation_accuracy_mean','validation_accuracy_std' ,'data_reduction_mean', 'data_reduction_std']]
key_mean = df_mean.decision_point
y_mean = df_mean.validation_accuracy_mean
x_mean = df_mean.data_reduction_mean
y_mean_err = df_mean.validation_accuracy_std
x_mean_err = df_mean.data_reduction_std


## Random error plot ## 

df_random = results[['decision_point','data_random_reduction_mean','data_random_accuracy_mean', 'data_random_accuracy_std']]
# df_random= df_random.iloc[1:].reset_index(drop=True)
key_random = df_random.decision_point
y_random = df_random.data_random_accuracy_mean
x_random = df_random.data_random_reduction_mean
y_random_err = df_random.data_random_accuracy_std


# ## CIS best
# # acc_col = 'validation_accuracy_best_policy'
# # size_col = 'data_reduction_best_policy'
# df_1 = results[['decision_point','validation_accuracy_best_policy','data_reduction_best_policy']]
# df_1  = df_1[df_1.decision_point.isin(relevant_decision_point_best)].reset_index(drop=True)
# key1  = df_1.decision_point
# y1 = df_1.validation_accuracy_best_policy
# x1 = df_1.data_reduction_best_policy

## CIS sample 
# acc_col = 'XGboost_Model_Accuracy'
# size_col = 'Data_Reduction'
df_2 = best_results[['index1','decision_point','XGboost_Model_Accuracy','Data_Reduction','frac']]
df_2  = df_2[df_2.index1.isin(relevant_decision_point_all_final)].reset_index(drop=True)
key2  = df_2.decision_point.astype(str) + '-' + df_2.frac.astype(str) 
y2 = df_2.XGboost_Model_Accuracy
x2 = df_2.Data_Reduction


# df_1_line =df_1[df_1.index.isin(relevant_decision_point_best)]
# df1_not_in_line=df_1[~df_1.index.isin(relevant_decision_point_best)]
                 


############################# Plot setting #########################################

fig, ax = plt.subplots(1, figsize=(30, 50))

## existing algorithms 
sns.regplot(data=df, x="x", y="y", fit_reg=False, marker="o", color="red", scatter_kws={'s':150}, scatter=True, ax=ax , label='exists methods')
## CIS errors 
plt.errorbar(x_mean, y_mean, xerr=x_mean_err, yerr=y_mean_err ,color='black', ecolor='lightgray', elinewidth=3, capsize=0, label='CIS_avg_policy') #fmt ='o',
## Random errors 
plt.errorbar(x_random, y_random, yerr=y_random_err ,color='green', ecolor='limegreen', elinewidth=3, capsize=0, label='random_avg_policy') #fmt ='o'

# # ## CIS best
# sns.regplot(data=df_1, y="validation_accuracy_best_policy", x="data_reduction_best_policy", fit_reg=False, marker="o", color="darkblue", scatter_kws={'s':150, 'facecolors':'none'}, ax=ax, label='CIS_best_policy')
# # sns.lineplot(y="validation_accuracy_best_policy", x="data_reduction_best_policy", markers=True,  color="darkblue", data=df_1, ax=ax)


# # ## CIS sample
sns.regplot(data=df_2, y="XGboost_Model_Accuracy", x="Data_Reduction", fit_reg=False, marker="o", color="blueviolet", scatter_kws={'s':100, }, ax=ax, label='CIS_best_policy')
sns.lineplot(y="XGboost_Model_Accuracy", x="Data_Reduction", markers=True,  color="blueviolet", data=df_2, ax=ax)

# sns.regplot(data=df1_not_in_line, y="validation_accuracy_best_policy", x="data_reduction_best_policy", fit_reg=False, marker="o", color="royalblue", scatter_kws={'s':150, 'facecolors':'none'}, ax=ax)

# for i in df_1_line.reset_index(drop=True).index[:-1]:
#     sns.lineplot(y="XGboost_Model_Accuracy", x="Data_Reduction", markers=True,  color="darkblue", data=df_1_line.reset_index(drop=True).iloc[i:i+2], ax=ax)
#     ax.lines[i+1].set_linestyle("--")



## labeling points 

for i, txt in enumerate(key):
    plt.annotate(txt, (x[i], y[i]),  textcoords="offset points", # how to position the text
                 xytext=(20,10), color='red', # distance from text to points (x,y)
                 ha='center', size=10)

for i, txt in enumerate(key_mean): 
    plt.annotate(txt, (x_mean[i], y_mean[i]),  textcoords="offset points",# how to position the text
                     xytext=(-10,0), color='black', # distance from text to points (x,y)
                     ha='center', size=6)

for i, txt in enumerate(key_random): 
    plt.annotate(txt, (x_random[i], y_random[i]),  textcoords="offset points",# how to position the text
                     xytext=(10,0), color='green', # distance from text to points (x,y)
                     ha='center', size=6)

# for i, txt in enumerate(key1): 
#     plt.annotate(txt, (x1[i], y1[i]),  textcoords="offset points",# how to position the text
#                      xytext=(0,-5),color='blue', # distance from text to points (x,y)
#                      ha='center', size=10)

    
for i, txt in enumerate(key2): 
    plt.annotate(txt, (x2[i], y2[i]),  textcoords="offset points",# how to position the text
                     xytext=(0,8),color='purple', # distance from text to points (x,y)
                     ha='center', size=6)

    
# for i, txt in enumerate(key_CIS_sampling): 
#     plt.annotate(txt, (x_CIS_sampling[i], y_CIS_sampling[i]),  textcoords="offset points",# how to position the text
#                      xytext=(0,-5),color='purple', # distance from text to points (x,y)
#                      ha='center', size=10)
    
    
## visulization
acc_min = round(min(list(df_mean.validation_accuracy_mean - df_mean.validation_accuracy_std) + list(df_random.data_random_accuracy_mean - df_random.data_random_accuracy_std) + list(df_2.XGboost_Model_Accuracy))-0.005 , 2)
acc_max = round(max(list(df_mean.validation_accuracy_mean + df_mean.validation_accuracy_std) + list(df_random.data_random_accuracy_mean + df_random.data_random_accuracy_std ) + list(df_2.XGboost_Model_Accuracy)) +0.005, 2)

plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel('Validation accuracy', fontsize=20)
plt.xlabel('Data Reduction', fontsize=20)        
plt.grid(axis='both',which='both')
plt.yticks(np.arange(acc_min, acc_max, step=0.005))
plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.legend(bbox_to_anchor=(0.5, 0.2),fontsize=18, facecolor='white', framealpha=1)
           
## Save plot 
plt.savefig('results/' + 'figure_' + file + '_Q_matrix_plot_by_trash.png')
# 

NameError: name 'Results' is not defined

## Test data results for decsition points

### Load data

In [25]:
train_data = train_data.append(data_val)

In [ ]:
train_data.groupby("Batch_Index").Batch_Index.count()

In [ ]:
# data_test  = data_test.rename(columns={"Cover_Type":"label"})

### LDIS, ENN, DROP

In [ ]:
## LDIS 
if file =='Wine':
    data_0 = train_data[train_data['quality']==0].reset_index() #wine
elif file =='mushrooms':
    data_0 = train_data[train_data['class']==0].reset_index() #Mushrooms
elif file =='adult':
    data_0 = train_data[train_data['income']==0].reset_index() #adult: 0,1
elif file =='diabetic_data':
    data_0 = train_data[train_data['readmitted']==0].reset_index() #diabetic_data: 0,1,2
elif file =='covtype':
    data_0 = train_data[train_data['label']==1].reset_index() #Covtype: 1,2,3,4,5,6,7
    
data_0_LDIS = instance_selection_numpy(data_0,k)

if file =='Wine':
    data_1 = train_data[train_data['quality']==1].reset_index() #wine
elif file =='mushrooms':
    data_1 = train_data[train_data['class']==1].reset_index() #Mushrooms
elif file =='adult':
    data_1 = train_data[train_data['income']==1].reset_index() #adult: 0,1
elif file =='diabetic_data':
    data_1 = train_data[train_data['readmitted']==1].reset_index() #diabetic_data: 0,1,2
elif file =='covtype':
    data_1 = train_data[train_data['label']==2].reset_index()  #Covtype: 1,2,3,4,5,6,7
    
data_1_LDIS = instance_selection_numpy(data_1,k)

if file =='diabetic_data':
    data_2 = train_data[train_data['readmitted']==2].reset_index() #diabetic_data: 0,1,2
    data_2_LDIS = instance_selection_numpy(data_2,k)

if file =='covtype':
    data_2 = train_data[train_data['label']==3].reset_index() #Covtype: 1,2,3,4,5,6,7
    data_2_LDIS = instance_selection_numpy(data_2,k)
    data_3 = train_data[train_data['label']==4].reset_index() #Covtype: 1,2,3,4,5,6,7
    data_3_LDIS = instance_selection_numpy(data_3,k)
    data_4 = train_data[train_data['label']==5].reset_index() #Covtype: 1,2,3,4,5,6,7
    data_4_LDIS = instance_selection_numpy(data_4,k)
    data_5 = train_data[train_data['label']==6].reset_index() #Covtype: 1,2,3,4,5,6,7
    data_5_LDIS = instance_selection_numpy(data_5,k)
    data_6 = train_data[train_data['label']==7].reset_index() #Covtype: 1,2,3,4,5,6,7
    data_6_LDIS = instance_selection_numpy(data_6,k)
        
    
data_LDIS = data_0_LDIS.append(data_1_LDIS, ignore_index=True)

if file =='diabetic_data':
    data_LDIS = data_LDIS.append(data_2_LDIS, ignore_index=True)

if file =='covtype':
    data_LDIS = data_LDIS.append(data_2_LDIS, ignore_index=True)
    data_LDIS = data_LDIS.append(data_3_LDIS, ignore_index=True)
    data_LDIS = data_LDIS.append(data_4_LDIS, ignore_index=True)
    data_LDIS = data_LDIS.append(data_5_LDIS, ignore_index=True)
    data_LDIS = data_LDIS.append(data_6_LDIS, ignore_index=True)
       
del data_LDIS['index']
data_LDIS.head()

In [31]:
# data_LDIS.to_csv(location + '/' + file + '_Train_Val_Data_LDIS.csv')
# data_LDIS = pd.read_csv(location + '/' + file + '_Train_Val_Data_LDIS.csv')
del data_LDIS['Unnamed: 0']
data_LDIS.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Batch_Index,label
0,3193,133,8,42,0,2250,234,236,134,1602,...,0,0,0,0,0,0,0,0,81,0
1,3193,333,17,85,17,5207,180,216,175,949,...,0,0,0,0,0,0,0,0,58,0
2,3179,333,20,272,85,2574,169,209,177,780,...,0,0,0,0,0,0,0,0,13,0
3,3290,93,5,90,22,5742,227,233,140,703,...,0,0,0,0,0,0,0,0,58,0
4,3155,271,22,845,234,3466,158,241,219,2170,...,0,0,0,0,0,0,0,0,29,0


In [ ]:
# data_test = data_test.rename(columns={'Cover_Type':'label'})

In [28]:

data_ENN = pd.read_csv(location + '/' + file + '_Train_Val_Data_ENN.csv', index_col=0)
# if file == 'Normal':
#     del data_ENN['X.1']
# if file == 'mushrooms':
#     del data_ENN['X']
# if file == 'Iris':
#     del data_ENN['X']
    
if file == 'Wine': 
    data_ENN = data_ENN.rename(columns={'fixed.acidity': 'fixed acidity', 'volatile.acidity': 'volatile acidity'
                                       ,'citric.acid': 'citric acid','residual.sugar': 'residual sugar'
                                       ,'free.sulfur.dioxide': 'free sulfur dioxide','total.sulfur.dioxide': 'total sulfur dioxide'})

if file == 'mushrooms':
    data_ENN = data_ENN.rename(columns={'cap.shape': 'cap-shape', 'cap.surface': 'cap-surface'
                                       ,'cap.color': 'cap-color','gill.attachment': 'gill-attachment'
                                       ,'gill.spacing':'gill-spacing', 'gill.size':'gill-size',
                                        'gill.color':'gill-color', 'stalk.shape':'stalk-shape',
                                        'stalk.root':'stalk-root','stalk.surface.above.ring':'stalk-surface-above-ring',
                                        'stalk.surface.below.ring':'stalk-surface-below-ring','stalk.color.above.ring':'stalk-color-above-ring',
                                        'stalk.color.below.ring':'stalk-color-below-ring',
                                        'veil.type':'veil-type','veil.color':'veil-color',
                                        'ring.number':'ring-number','ring.type':'ring-type',
                                        'spore.print.color':'spore-print-color',})

        
if file == 'diabetic_data':
    data_ENN = data_ENN.rename(columns={'glyburide.metformin': 'glyburide-metformin', 
                                        'glipizide.metformin':'glipizide-metformin', 
                                        'glimepiride.pioglitazone':'glimepiride-pioglitazone', 
                                        'metformin.rosiglitazone':'metformin-rosiglitazone', 
                                        'metformin.pioglitazone': 'metformin-pioglitazone'})
data_ENN.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
1,1859,18,12,67,11,90,211,215,139,792,...,0,0,0,0,0,0,0,0,0,3
2,1860,18,13,95,15,90,210,213,138,780,...,0,0,0,0,0,0,0,0,0,3
3,1861,35,14,60,11,85,218,209,124,832,...,0,0,0,0,0,0,0,0,0,3
5,1866,23,14,85,16,108,212,210,133,819,...,0,0,0,0,0,0,0,0,0,3
6,1868,27,16,67,17,95,212,204,125,859,...,0,0,0,0,0,0,0,0,0,3


In [29]:
if file =='covtype':
    train_data["label"] = train_data.Cover_Type.map({1:0,2:1,3:2,4:3,5:4,6:5,7:6})
    data_val["label"] = data_val.Cover_Type.map({1:0,2:1,3:2,4:3,5:4,6:5,7:6})
    data_test["label"] = data_test.Cover_Type.map({1:0,2:1,3:2,4:3,5:4,6:5,7:6})
    data_ENN["label"] = data_ENN.Cover_Type.map({1:0,2:1,3:2,4:3,5:4,6:5,7:6})
    data_LDIS["label"] = data_LDIS.label.map({1:0,2:1,3:2,4:3,5:4,6:5,7:6})
    del train_data["Cover_Type"]
    del data_val["Cover_Type"]
    del data_test["Cover_Type"]
    del data_ENN["Cover_Type"]

In [ ]:
# #DROP3
# data_DROP3 = pd.read_csv(location + '/' + file + '_Train_Val_Data_DROP3.csv', index_col=0)
# # if file == 'Normal':
# #     del data_DROP3['X.1']
# # if file == 'mushrooms':
# #     del data_DROP3['X']
# # if file == 'Iris':
# #     del data_DROP3['X']
    
# if file == 'Wine': 
#     data_DROP3 = data_DROP3.rename(columns={'fixed.acidity': 'fixed acidity', 'volatile.acidity': 'volatile acidity'
#                                        ,'citric.acid': 'citric acid','residual.sugar': 'residual sugar'
#                                        ,'free.sulfur.dioxide': 'free sulfur dioxide','total.sulfur.dioxide': 'total sulfur dioxide'})
# if file == 'mushrooms':
#     data_DROP3 = data_DROP3.rename(columns={'cap.shape': 'cap-shape', 'cap.surface': 'cap-surface'
#                                        ,'cap.color': 'cap-color','gill.attachment': 'gill-attachment'
#                                        ,'gill.spacing':'gill-spacing', 'gill.size':'gill-size',
#                                         'gill.color':'gill-color', 'stalk.shape':'stalk-shape',
#                                         'stalk.root':'stalk-root','stalk.surface.above.ring':'stalk-surface-above-ring',
#                                         'stalk.surface.below.ring':'stalk-surface-below-ring','stalk.color.above.ring':'stalk-color-above-ring',
#                                         'stalk.color.below.ring':'stalk-color-below-ring',
#                                         'veil.type':'veil-type','veil.color':'veil-color',
#                                         'ring.number':'ring-number','ring.type':'ring-type',
#                                         'spore.print.color':'spore-print-color',})

# data_DROP3.head()

In [33]:
Test_Results = []

X_test,y_test=data_separate(data_test,0)


# X_test,y_test=data_separate(data_val,0)

print ("------------ALL------------------")
X_train,y_train=data_separate(train_data,1)
accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')

Test_Results.append(['All',1, 'All',accuracy, 1-(len(train_data)/len(train_data))])

print ("------------ENN------------------")
X_train,y_train=data_separate(data_ENN,0)
accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')
Test_Results.append(['ENN',1,'ENN',accuracy,1-(len(data_ENN)/len(train_data))])

# print ("------------DROP3------------------")
# X_train,y_train=data_separate(data_DROP3,0)
# accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')

# Test_Results.append(['DROP3',1,'DROP3',accuracy,1-(len(data_DROP3)/len(train_data))])

print ("------------LDIS ------------------")
X_train,y_train=data_separate(data_LDIS ,1)
accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')

Test_Results.append(['LDIS',1,'LDIS',accuracy,1-(len(data_LDIS)/len(train_data))])


------------LDIS ------------------
accuracy: 0.6704990404722769
run time: 32.031217800000036


In [34]:
Test_Results

[['All', 1, 'All', 0.6713079696737606, 0.0],
 ['ENN', 1, 'ENN', 0.6720480538368201, 0.06656067330882143],
 ['LDIS', 1, 'LDIS', 0.6704990404722769, 0.8782854893085117]]

### Decision points & selected sampled decision points results

In [ ]:
# include_results = pareto[pareto.decision_point.isin(relevant_decision_point_best)]

In [ ]:
# include_results_sampling = sampling_CIS_table[["decision_point","policy","CIS_Batches","frac"]]

In [ ]:
# import ast

# for index, include in include_results.best_policy.items():
#     print(include)
#     decision_point = int(include_results[include_results.best_policy == include].decision_point)
#     print(decision_point)
#     data_CIS = train_data[train_data.Batch_Index.isin(ast.literal_eval(include))]
#     X_train,y_train=data_separate(data_CIS ,1)
#     if len(X_train)==0:
#         accuracy=0
#     else:
#         accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')
#     size = 1-(len(data_CIS)/len(train_data))
# #     effeicency = round((accuracy/max_accuracy_validation) * (size), 3)
#     Test_Results.append([decision_point,1, "CIS Batches = " + str(include), accuracy,size])        

In [35]:
import ast

for index, row in pareto.iterrows():
    include = row.policy
    frac = row.frac
    print(include)
    print(frac)
    data_CIS = train_data[train_data.Batch_Index.isin(ast.literal_eval(include))]
    data_CIS = data_CIS.groupby('Batch_Index', group_keys=False).apply(pd.DataFrame.sample, frac=frac)
    X_train,y_train=data_separate(data_CIS ,1)
    if len(X_train)==0:
        accuracy=0
    else:
        accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')
    size = 1-(len(data_CIS)/len(train_data))
    Test_Results.append([row.decision_point, frac, "CIS Batches = " + str(include) + ' frac = ' + str(frac) , accuracy,size])   

['33']
1.0
accuracy: 0.4888428009603883
run time: 2.4856048999999985
['33']
0.5
accuracy: 0.4888428009603883
run time: 2.458903300000088
['0', '33']
0.5
accuracy: 0.6242351746512569
run time: 4.4926368999999795
['0', '1', '13', '33', '5', '61', '62']
0.5
accuracy: 0.6661704086813593
run time: 18.205862300000035
['1', '13', '16', '19', '28', '33', '5', '51', '58', '61', '74']
0.5
accuracy: 0.6691307453335973
run time: 29.46053219999999
['0', '1', '13', '16', '19', '28', '33', '5', '51', '58', '60', '61', '66', '74', '77']
0.5
accuracy: 0.6738552360954536
run time: 40.16177159999995
['0', '1', '10', '12', '13', '15', '16', '19', '20', '28', '3', '33', '43', '44', '46', '47', '50', '51', '57', '58', '60', '61', '62', '66', '74', '77', '8', '84']
0.5
accuracy: 0.6748965173016187
run time: 65.94557200000008
['0', '1', '10', '12', '13', '15', '16', '19', '20', '28', '3', '33', '43', '44', '46', '47', '5', '50', '51', '57', '58', '59', '60', '61', '62', '66', '74', '77', '8', '83', '84']
0.5


accuracy: 0.6711186458180942
run time: 220.40498730000036
['0', '1', '10', '12', '13', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '27', '28', '29', '3', '30', '31', '32', '33', '35', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '9']
1.0
accuracy: 0.6714886878996239
run time: 221.1941643
['0', '1', '10', '12', '13', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '37', '38', '39', '4', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77

In [36]:
Test_Results

[['All', 1, 'All', 0.6713079696737606, 0.0],
 ['ENN', 1, 'ENN', 0.6720480538368201, 0.06656067330882143],
 ['LDIS', 1, 'LDIS', 0.6704990404722769, 0.8782854893085117],
 [1.0,
  1.0,
  "CIS Batches = ['33'] frac = 1.0",
  0.4888428009603883,
  0.9999935457359905],
 [1.0,
  0.5,
  "CIS Batches = ['33'] frac = 0.5",
  0.4888428009603883,
  0.999995697157327],
 [2.0,
  0.5,
  "CIS Batches = ['0', '33'] frac = 0.5",
  0.6242351746512569,
  0.9935414331478091],
 [7.0,
  0.5,
  "CIS Batches = ['0', '1', '13', '33', '5', '61', '62'] frac = 0.5",
  0.6661704086813593,
  0.9342955923831079],
 [11.0,
  0.5,
  "CIS Batches = ['1', '13', '16', '19', '28', '33', '5', '51', '58', '61', '74'] frac = 0.5",
  0.6691307453335973,
  0.8906002250386718],
 [15.0,
  0.5,
  "CIS Batches = ['0', '1', '13', '16', '19', '28', '33', '5', '51', '58', '60', '61', '66', '74', '77'] frac = 0.5",
  0.6738552360954536,
  0.8482279818161869],
 [28.0,
  0.5,
  "CIS Batches = ['0', '1', '10', '12', '13', '15', '16', '19',

In [37]:
Test_Results_Table = pd.DataFrame(Test_Results, columns =['decision_point', 'frac', 'IS Method', 'XGboost Model Accuracy', 'Data Reduction'])


In [38]:
max_accuracy_test = Test_Results_Table['XGboost Model Accuracy'].max()

In [39]:
max_accuracy_test

0.6773749386848876

In [40]:
pareto

,Unnamed: 0,decision_point,policy,frac,XGboost_Model_Accuracy,Data_Reduction,effeicency,name,index1
0,0,1.0,['33'],1.0,0.488000,1.000000,0.724036,1.0 frac=1.0,1
1,1,1.0,['33'],0.5,0.488264,0.999995,0.724000,1.0 frac=0.5,2
2,2,2.0,"['0', '33']",0.5,0.628859,0.993539,0.927000,2.0 frac=0.5,3
3,8,7.0,"['0', '1', '13', '33', '5', '61', '62']",0.5,0.665519,0.934391,0.923000,7.0 frac=0.5,9
4,14,11.0,"['1', '13', '16', '19', '28', '33', '5', '51',...",0.5,0.668617,0.890619,0.884000,11.0 frac=0.5,15
5,22,15.0,"['0', '1', '13', '16', '19', '28', '33', '5', ...",0.5,0.670489,0.848165,0.844000,15.0 frac=0.5,23
6,38,28.0,"['0', '1', '10', '12', '13', '15', '16', '19',...",0.5,0.673200,0.741681,0.741000,28.0 frac=0.5,39
7,42,31.0,"['0', '1', '10', '12', '13', '15', '16', '19',...",0.5,0.673716,0.721001,0.721000,31.0 frac=0.5,43
8,51,35.0,"['0', '1', '10', '12', '13', '15', '16', '19',...",0.5,0.673824,0.669907,0.670000,35.0 frac=0.5,52
9,121,35.0,"['0', '1', '10', '12', '13', '15', '16', '19',...",1.0,0.674000,0.340000,0.340000,35.0 frac=1.0,122


In [41]:
Test_Results_Table["Effectiveness"] = (Test_Results_Table['XGboost Model Accuracy']/max_accuracy_test)* Test_Results_Table['Data Reduction'] 

Test_Results_Table

,decision_point,frac,IS Method,XGboost Model Accuracy,Data Reduction,Effectiveness
0,All,1.0,All,0.671308,0.000000,0.000000
1,ENN,1.0,ENN,0.672048,0.066561,0.066037
2,LDIS,1.0,LDIS,0.670499,0.878285,0.869370
3,1,1.0,CIS Batches = ['33'] frac = 1.0,0.488843,0.999994,0.721668
4,1,0.5,CIS Batches = ['33'] frac = 0.5,0.488843,0.999996,0.721669
5,2,0.5,"CIS Batches = ['0', '33'] frac = 0.5",0.624235,0.993541,0.915599
6,7,0.5,"CIS Batches = ['0', '1', '13', '33', '5', '61'...",0.666170,0.934296,0.918841
7,11,0.5,"CIS Batches = ['1', '13', '16', '19', '28', '3...",0.669131,0.890600,0.879761
8,15,0.5,"CIS Batches = ['0', '1', '13', '16', '19', '28...",0.673855,0.848228,0.843821
9,28,0.5,"CIS Batches = ['0', '1', '10', '12', '13', '15...",0.674897,0.741670,0.738957


### Random results

In [42]:

random_test = pd.DataFrame()
random_test['data_random_reduction'] = Test_Results_Table['Data Reduction']
random_test['data_random_accuracy_mean'] = 0
# random_test['data_random_accuracy_std'] = 0
random_test['data_random_accuracy_min']= 0
# random_test['data_random_accuracy_max']= 0
random_test['data_random_accuracy_percentile_95']= 0
random_test['data_random_accuracy_percentile_90']= 0


In [43]:
point_results = {}
for i in random_test.index:
    print ("results for index {}".format(i))
    res = []
    data_frac = (1 - random_test.iloc[i].values[0])
    if data_frac == 0 :
        data_frac =  0.001
    for j in range(30):
        train_data_sample = train_data.sample(frac=data_frac)
        X_train,y_train=data_separate(train_data_sample,1)
        accuracy = Learner(X_train, X_test, y_train, y_test, 'XGB')
        res.append(accuracy)
    point_results[i] = res

results for index 0
accuracy: 0.6713079696737606
run time: 266.84009349999906
accuracy: 0.6713079696737606
run time: 262.82846170000084
accuracy: 0.6713079696737606
run time: 256.6622987999999
accuracy: 0.6713079696737606
run time: 241.1794131000006
accuracy: 0.6713079696737606
run time: 238.79357889999847
accuracy: 0.6713079696737606
run time: 241.39436249999926
accuracy: 0.6713079696737606
run time: 249.56426310000097
accuracy: 0.6713079696737606
run time: 229.36178269999982
accuracy: 0.6713079696737606
run time: 267.5707975000005
accuracy: 0.6713079696737606
run time: 273.7663822000013
accuracy: 0.6713079696737606
run time: 237.2648277999997
accuracy: 0.6713079696737606
run time: 240.47983899999963
accuracy: 0.6713079696737606
run time: 237.29926290000003
accuracy: 0.6713079696737606
run time: 230.52428899999904
accuracy: 0.6713079696737606
run time: 250.46796899999936
accuracy: 0.6713079696737606
run time: 252.2238139000001
accuracy: 0.6713079696737606
run time: 268.6563833
accurac

accuracy: 0.3639406899993976
run time: 2.2739540000002307
accuracy: 0.3639406899993976
run time: 2.412438600000314
accuracy: 0.3639406899993976
run time: 2.3828848999983165
accuracy: 0.3639406899993976
run time: 2.3773833999985072
accuracy: 0.4888428009603883
run time: 2.470850200003042
accuracy: 0.3639406899993976
run time: 2.479808799998864
accuracy: 0.4888428009603883
run time: 2.3833234000012453
accuracy: 0.4888428009603883
run time: 2.3893697999992582
accuracy: 0.3639406899993976
run time: 2.3677520000019285
results for index 5
accuracy: 0.661970861337487
run time: 5.853649600001518
accuracy: 0.6603530029345198
run time: 5.802388299998711
accuracy: 0.6663167043880106
run time: 5.921413699998084
accuracy: 0.6664027606860408
run time: 5.952575600000273
accuracy: 0.6633993958847878
run time: 5.855023500000243
accuracy: 0.6646988459850434
run time: 5.796334399998159
accuracy: 0.6636317478894693
run time: 5.855150799998228
accuracy: 0.6612824109532456
run time: 5.783591299998079
accura

accuracy: 0.671531716048639
run time: 1039.110770899999
accuracy: 0.6720738707262292
run time: 36796.2748098
accuracy: 0.6715833498274572
run time: 64.82596889999695
accuracy: 0.6718415187215476
run time: 60.81947390000278
accuracy: 0.6721168988752442
run time: 63.17435609998938
accuracy: 0.6721513213944562
run time: 62.281454900003155
accuracy: 0.6712563358949425
run time: 64.90866010000173
accuracy: 0.6706883643279433
run time: 56.88279319999856
accuracy: 0.6716694061254873
run time: 59.30143300000054
accuracy: 0.6710325895200641
run time: 57.1458241000073
accuracy: 0.6722029551732743
run time: 59.81133900000714
accuracy: 0.6715231104188361
run time: 55.99029750000045
accuracy: 0.6709895613710489
run time: 59.2686226000078
accuracy: 0.6710756176690791
run time: 56.10291350001353
accuracy: 0.6722287720626834
run time: 59.24018849999993
accuracy: 0.6715231104188361
run time: 56.035686599992914
accuracy: 0.6722029551732743
run time: 59.48039709999284
results for index 10
accuracy: 0.672

accuracy: 0.6711358570777002
run time: 183.35462369999732
accuracy: 0.6719017581301687
run time: 173.0480685999937
accuracy: 0.6712219133757305
run time: 177.26485200000752
accuracy: 0.6719361806493809
run time: 178.6070805000054
accuracy: 0.6709637444816399
run time: 171.39532700000564
accuracy: 0.6715661385678511
run time: 165.1968729999935
accuracy: 0.6714800822698209
run time: 174.3195343999978
accuracy: 0.6716952230148964
run time: 171.32079740001063
accuracy: 0.6712391246353364
run time: 168.4392102000129
accuracy: 0.671023983890261
run time: 183.79760929998884
accuracy: 0.6706023080299132
run time: 192.8580423999956
accuracy: 0.6713165753035636
run time: 180.73822069999005
accuracy: 0.6717812793129265
run time: 189.48669960000552
accuracy: 0.6722115608030774
run time: 184.11944490000315
accuracy: 0.6707744206259735
run time: 183.68937170000572
accuracy: 0.6713079696737606
run time: 167.17190780000237
accuracy: 0.6717296455341084
run time: 165.43024739999964
accuracy: 0.671557532

accuracy: 0.6719533919089868
run time: 236.18163639999693
accuracy: 0.6717296455341084
run time: 206.48248130000138
accuracy: 0.6716091667168662
run time: 207.79382770000666
accuracy: 0.6714886878996239
run time: 203.00292700000864
results for index 19
accuracy: 0.6714198428611998
run time: 50117.89034700001
accuracy: 0.6715747441976541
run time: 273.7705860999995
accuracy: 0.6712907584141545
run time: 239.1762071000121
accuracy: 0.671548927308245
run time: 219.8383257999958
accuracy: 0.6716608004956843
run time: 241.01618079998298
accuracy: 0.6714284484910028
run time: 234.10879790000035
accuracy: 0.672005025687805
run time: 235.6740488000214
accuracy: 0.6704560123232619
run time: 238.597927899973
accuracy: 0.6717210399043054
run time: 214.83489619998727
accuracy: 0.6716780117552903
run time: 206.78312880001613
accuracy: 0.6716694061254873
run time: 207.80634949999512
accuracy: 0.6713768147121847
run time: 205.64255049999338
accuracy: 0.6716866173850933
run time: 208.0231195000233
acc

accuracy: 0.6710842232988822
run time: 199.43561190000037
accuracy: 0.6719017581301687
run time: 199.87324770001578
accuracy: 0.6711788852267153
run time: 202.74004299999797
accuracy: 0.6714886878996239
run time: 200.17504780000309
accuracy: 0.6717984905725326
run time: 201.5893078999943
accuracy: 0.6713337865631697
run time: 198.99454290000722
accuracy: 0.6715919554572601
run time: 198.47230900000432
accuracy: 0.6717726736831235
run time: 199.26173420000123
accuracy: 0.6713079696737606
run time: 199.01836380001623
accuracy: 0.6716780117552903
run time: 199.50282639998477
accuracy: 0.6719878144281989
run time: 199.05628119999892
accuracy: 0.671531716048639
run time: 200.58529440002167
accuracy: 0.6720566594666231
run time: 198.84224490000634
results for index 24
accuracy: 0.6721943495434713
run time: 200.30139450001298
accuracy: 0.6708346600345946
run time: 199.9784299999883
accuracy: 0.6716005610870631
run time: 199.7903263999906
accuracy: 0.6717984905725326
run time: 199.209570000006

accuracy: 0.671540321678442
run time: 213.3177234000177
accuracy: 0.6724697296971679
run time: 216.63633189996472
accuracy: 0.6708518712942007
run time: 239.17100329999812
accuracy: 0.6723234339905166
run time: 251.66978470003232
accuracy: 0.6712993640439575
run time: 255.13155940000433
accuracy: 0.6718415187215476
run time: 230.65579320001416
accuracy: 0.6728569830383037
run time: 212.61807769996813
accuracy: 0.6713337865631697
run time: 211.2074680000078
accuracy: 0.6716608004956843
run time: 211.1194048000034
accuracy: 0.6714284484910028
run time: 213.85535279999021
accuracy: 0.671548927308245
run time: 209.99090089998208
accuracy: 0.672495546586577
run time: 209.4802963999682
accuracy: 0.6721771382838653
run time: 210.5455503000412
accuracy: 0.6714456597506089
run time: 210.7185624000267
accuracy: 0.6719706031685929
run time: 211.69464900001185
accuracy: 0.6719189693897748
run time: 213.32553339999868
accuracy: 0.6713337865631697
run time: 31123.059031700017
accuracy: 0.67139402597

In [44]:
for i in random_test.index:
    random_test['data_random_accuracy_mean'].iloc[i] = np.mean(point_results[i])
#     random_test['data_random_accuracy_std'].iloc[i] = np.std(point_results[i])
    random_test['data_random_accuracy_min'].iloc[i] = np.min(point_results[i])
#     random_test['data_random_accuracy_max'].iloc[i] = np.max(point_results[i])
#     random_test['data_random_accuracy_percentile_95'].iloc[i]= np.percentile(point_results[i],95)
#     random_test['data_random_accuracy_percentile_90'].iloc[i]= np.percentile(point_results[i],90)
    


C:\Users\mmoran1\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [45]:
Test_Results_Table = pd.concat([Test_Results_Table, random_test], axis=1)
Test_Results_Table.head()

,decision_point,frac,IS Method,XGboost Model Accuracy,Data Reduction,Effectiveness,data_random_reduction,data_random_accuracy_mean,data_random_accuracy_min,data_random_accuracy_percentile_95,data_random_accuracy_percentile_90
0,All,1.0,All,0.671308,0.000000,0.000000,0.000000,0.671308,0.671308,0,0
1,ENN,1.0,ENN,0.672048,0.066561,0.066037,0.066561,0.671692,0.670852,0,0
2,LDIS,1.0,LDIS,0.670499,0.878285,0.869370,0.878285,0.671696,0.670017,0,0
3,1,1.0,CIS Batches = ['33'] frac = 1.0,0.488843,0.999994,0.721668,0.999994,0.443045,0.363941,0,0
4,1,0.5,CIS Batches = ['33'] frac = 0.5,0.488843,0.999996,0.721669,0.999996,0.405575,0.363941,0,0


In [46]:
from scipy import stats
Test_Results_Table['CIS_score_precentile_in_random'] = 0
prec = []
for i in Test_Results_Table.index:
    prec.append(stats.percentileofscore(point_results[i]+ [Test_Results_Table['XGboost Model Accuracy'].iloc[i]] , Test_Results_Table['XGboost Model Accuracy'].iloc[i]))

In [47]:
Test_Results_Table['CIS_score_precentile_in_random'] = prec

In [48]:
Test_Results_Table

,decision_point,frac,IS Method,XGboost Model Accuracy,Data Reduction,Effectiveness,data_random_reduction,data_random_accuracy_mean,data_random_accuracy_min,data_random_accuracy_percentile_95,data_random_accuracy_percentile_90,CIS_score_precentile_in_random
0,All,1.0,All,0.671308,0.000000,0.000000,0.000000,0.671308,0.671308,0,0,51.612903
1,ENN,1.0,ENN,0.672048,0.066561,0.066037,0.066561,0.671692,0.670852,0,0,80.645161
2,LDIS,1.0,LDIS,0.670499,0.878285,0.869370,0.878285,0.671696,0.670017,0,0,6.451613
3,1,1.0,CIS Batches = ['33'] frac = 1.0,0.488843,0.999994,0.721668,0.999994,0.443045,0.363941,0,0,69.354839
4,1,0.5,CIS Batches = ['33'] frac = 0.5,0.488843,0.999996,0.721669,0.999996,0.405575,0.363941,0,0,83.870968
5,2,0.5,"CIS Batches = ['0', '33'] frac = 0.5",0.624235,0.993541,0.915599,0.993541,0.662413,0.657900,0,0,3.225806
6,7,0.5,"CIS Batches = ['0', '1', '13', '33', '5', '61'...",0.666170,0.934296,0.918841,0.934296,0.671051,0.668451,0,0,3.225806
7,11,0.5,"CIS Batches = ['1', '13', '16', '19', '28', '3...",0.669131,0.890600,0.879761,0.890600,0.671487,0.669707,0,0,3.225806
8,15,0.5,"CIS Batches = ['0', '1', '13', '16', '19', '28...",0.673855,0.848228,0.843821,0.848228,0.671495,0.669828,0,0,100.000000
9,28,0.5,"CIS Batches = ['0', '1', '10', '12', '13', '15...",0.674897,0.741670,0.738957,0.741670,0.671809,0.670688,0,0,100.000000


In [49]:
# Test_Results_Table['data_random_mean_effectiveness'] = Test_Results_Table['data_random_accuracy_mean']*Test_Results_Table["data_random_reduction"]

In [50]:
# Test_Results_Table.to_csv("results/" + file + "_Test_Results_Table.csv")
Test_Results_Table.to_excel("results/" + file + "_Test_Results_Table.xlsx")

In [ ]:
### 